In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

In [2]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [4]:
display(links.head(), movies.head(), ratings.head(), tags.head())

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [6]:
display(len(links), len(movies), len(ratings), len(tags))

9742

9742

100836

3683

##### Найдем лучшие фильмы по суммарному рейтингу

In [56]:
top5 = ratings.groupby('movieId').sum().sort_values('rating', ascending=False).head(20)
top5

,userId,rating,timestamp
movieId,,,
318,95829,1404.0,376924839127
356,101385,1370.0,386165236681
296,90621,1288.5,349204311001
2571,85236,1165.5,350270041779
593,85535,1161.0,320035674330
260,76484,1062.0,298509531263
110,73375,955.5,259448308280
2959,69737,931.5,282265436424
527,69481,929.5,257743980936


In [58]:
top5mov = pd.merge(top5, movies, how='left', on='movieId')
top5mov[['title','rating']]

,title,rating
0,"Shawshank Redemption, The (1994)",1404.0
1,Forrest Gump (1994),1370.0
2,Pulp Fiction (1994),1288.5
3,"Matrix, The (1999)",1165.5
4,"Silence of the Lambs, The (1991)",1161.0
5,Star Wars: Episode IV - A New Hope (1977),1062.0
6,Braveheart (1995),955.5
7,Fight Club (1999),931.5
8,Schindler's List (1993),929.5
9,Jurassic Park (1993),892.5


##### Найдем лучшие фильмы по метрике: сумма оценок умножить на количество пользовательсиких тэгов на фильм

In [60]:
top5 = ratings.groupby('movieId').sum()#.sort_values('rating', ascending=False)
top5mov = pd.merge(top5, movies, how='left', on='movieId')
top5sum = tags.groupby('movieId')[['tag']].count()#.sort_values('rating', ascending=False).head()
RS_top = pd.merge(top5mov, top5sum, how='left', on='movieId').fillna(0)
RS_top['new_rating'] = RS_top['rating'] * (1 + RS_top['tag'])
RS_top.sort_values('new_rating', ascending=False).reset_index()[['title','new_rating']].head(20)

,title,new_rating
0,Pulp Fiction (1994),234507.0
1,Fight Club (1999),51232.5
2,Star Wars: Episode IV - A New Hope (1977),28674.0
3,Léon: The Professional (a.k.a. The Professiona...,19242.0
4,Eternal Sunshine of the Spotless Mind (2004),19075.0
5,2001: A Space Odyssey (1968),17829.0
6,Inception (2010),15700.5
7,"Big Lebowski, The (1998)",13728.0
8,Forrest Gump (1994),13700.0
9,Donnie Darko (2001),13020.0


##### Найдем лучшие фильмы по той же метрике для фильмов в категориях Action и Comedy

In [68]:
RS_top[RS_top['genres'].str.find('Action')>=0].sort_values('new_rating', ascending=False).reset_index()\
            [['title','new_rating','genres']].head(20)

,title,new_rating,genres
0,Fight Club (1999),51232.5,Action|Crime|Drama|Thriller
1,Star Wars: Episode IV - A New Hope (1977),28674.0,Action|Adventure|Sci-Fi
2,Léon: The Professional (a.k.a. The Professiona...,19242.0,Action|Crime|Drama|Thriller
3,Inception (2010),15700.5,Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX
4,Braveheart (1995),10510.5,Action|Drama|War
5,Star Wars: Episode V - The Empire Strikes Back...,9784.5,Action|Adventure|Sci-Fi
6,"Lord of the Rings: The Return of the King, The...",8382.0,Action|Adventure|Drama|Fantasy
7,Terminator 2: Judgment Day (1991),8005.5,Action|Sci-Fi
8,Raiders of the Lost Ark (Indiana Jones and the...,7573.5,Action|Adventure
9,Blade Runner (1982),7119.0,Action|Sci-Fi|Thriller


In [69]:
RS_top[RS_top['genres'].str.find('Comedy')>=0].sort_values('new_rating', ascending=False).reset_index()\
            [['title','new_rating','genres']].head(20)

,title,new_rating,genres
0,Pulp Fiction (1994),234507.0,Comedy|Crime|Drama|Thriller
1,"Big Lebowski, The (1998)",13728.0,Comedy|Crime
2,Forrest Gump (1994),13700.0,Comedy|Drama|Romance|War
3,Dr. Strangelove or: How I Learned to Stop Worr...,4554.0,Comedy|War
4,Fargo (1996),4470.0,Comedy|Crime|Drama|Thriller
5,Toy Story (1995),3372.0,Adventure|Animation|Children|Comedy|Fantasy
6,Toy Story 2 (1999),3370.5,Adventure|Animation|Children|Comedy|Fantasy
7,Life Is Beautiful (La Vita è bella) (1997),3285.0,Comedy|Drama|Romance|War
8,American Pie (1999),3132.0,Comedy|Romance
9,Monty Python and the Holy Grail (1975),2830.0,Adventure|Comedy|Fantasy
